# 🔬 ViTHSD - Dataset A - PhoBERT (Standard Training)

**Model 1/3 cho Dataset A**

- **Model**: PhoBERT (vinai/phobert-base)
- **Dataset**: A (Original ViTHSD - chỉ content + labels)
- **Task**: Multi-Label Classification (11 labels)
- **Technique**: Standard Training (no KD, no CoT)

### 📌 Label Constraint
- Each category can only have **ONE level**: offensive OR hate (not both)

## Step 1: Setup Environment

In [1]:
# Cài đặt các thư viện cần thiết
!pip uninstall -y protobuf
!pip install -q protobuf==3.20.0
!pip install -q transformers datasets accelerate sentencepiece
!pip install -q scikit-learn pandas numpy tqdm openpyxl

# Suppress warnings
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'
warnings.filterwarnings('ignore')

print("✅ Dependencies installed!")

Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.0 which is incompatible.
google-cloud-secret-manager 2.25.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-vision 3.11.0 requires protobuf!=4.21.0,!=4.

In [2]:
# 🎮 GPU Auto-Detection & Optimization
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"\n{'='*70}")
    print(f"🎮 DETECTED: {num_gpus} GPU(s)")
    print(f"{'='*70}")
    
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
    
    # Auto-configure based on number of GPUs
    if num_gpus == 1:
        # Single GPU (P100)
        print(f"\n{'='*70}")
        print("⚙️  SINGLE GPU MODE (P100 Optimized)")
        print(f"{'='*70}")
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        
        BATCH_SIZE_PHOBERT = 12  # P100 (16GB)
        
        GPU_CONFIG = {
            'num_gpus': 1,
            'device': 'cuda:0',
            'batch_sizes': {
                'phobert': BATCH_SIZE_PHOBERT
            },
            'gradient_accumulation_steps': 2
        }
        
    elif num_gpus >= 2:
        # Multi-GPU (T4 x2)
        print(f"\n{'='*70}")
        print("⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)")
        print(f"{'='*70}")
        
        BATCH_SIZE_PHOBERT = 16  # T4 x2 (32GB)
        
        GPU_CONFIG = {
            'num_gpus': num_gpus,
            'device': 'cuda',
            'batch_sizes': {
                'phobert': BATCH_SIZE_PHOBERT
            },
            'gradient_accumulation_steps': 1
        }
    
    print(f"\n📊 Configured Batch Size:")
    print(f"   PhoBERT:   {GPU_CONFIG['batch_sizes']['phobert']}")
    print(f"\n{'='*70}\n")
    
else:
    print("❌ No GPU detected! This notebook requires GPU.")
    GPU_CONFIG = None

PyTorch version: 2.6.0+cu124
CUDA available: True

🎮 DETECTED: 2 GPU(s)
GPU 0: Tesla T4
GPU 1: Tesla T4

⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)

📊 Configured Batch Size:
   PhoBERT:   16




## Step 2: Import và Setup Paths

In [3]:
import sys
import os

# Copy files ra working directory
!cp -r /kaggle/input/vithsd-experiment/* /kaggle/working/

os.chdir("/kaggle/working")
sys.path.insert(0, "/kaggle/working")

print("📁 Files copied to working directory")

📁 Files copied to working directory


In [4]:
# Import modules
from config import FINAL_LABELS, DATA_DIR, OUTPUT_DIR
from data_preparation import load_dataset_A
from evaluation import Evaluator
from models import create_model

print("✅ All modules imported!")
print(f"📊 Labels: {len(FINAL_LABELS)}")

✅ All modules imported!
📊 Labels: 11


## Step 3: Load Dataset A

In [5]:
# Load Dataset A
train_texts, train_labels = load_dataset_A('train')
test_texts, test_labels = load_dataset_A('test')

print(f"📊 Dataset A:")
print(f"   Train: {len(train_texts)} samples")
print(f"   Test: {len(test_texts)} samples")

[ViHSDLoader] Loaded 7000 samples from train
[ViHSDLoader] Loaded 1800 samples from test
📊 Dataset A:
   Train: 7000 samples
   Test: 1800 samples


## Step 4: Train PhoBERT

In [6]:
# Initialize
evaluator = Evaluator()

# PhoBERT - Dataset A (Standard Training)
print("="*70)
print("🚀 PHOBERT - DATASET A (Standard)")
print("="*70)

model_phobert = create_model(
    "phobert", 
    dataset_type="A",
    batch_size=GPU_CONFIG['batch_sizes']['phobert'],  # Auto-adjusted
    num_epochs=3,
    learning_rate=2e-5
)

# Train
model_phobert.train(train_texts, train_labels)

# Predict
pred_phobert, _ = model_phobert.predict(test_texts)

# Evaluate
result = evaluator.evaluate(test_labels, pred_phobert, "PhoBERT", "A")
evaluator.print_result(result)


🚀 PHOBERT - DATASET A (Standard)
[PhoBERT_MultiLabel] Training...
  Device: cuda
  Original samples: 7000
  Mode: Standard Training (Dataset A)
  [Oversampling] Added 540 samples for minority labels


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

  Epoch 2: avg_loss = 0.5586


Epoch 3/3: 100%|██████████| 472/472 [05:47<00:00,  1.36it/s, loss=0.2548]


  Epoch 3: avg_loss = 0.4914
  ✓ Training completed


Predicting: 100%|██████████| 113/113 [00:20<00:00,  5.53it/s]


📊 PhoBERT on Dataset A (Multi-Label)
  Subset Accuracy (Exact Match): 0.5089
  Hamming Loss:                  0.1042
  F1 Samples:                    0.5801
  F1 Macro:                      0.2586
  F1 Micro:                      0.5412
  Precision (Samples):           0.5791
  Recall (Samples):              0.5993

  F1 per Label:
    normal                   : 0.7531
    individuals#offensive    : 0.0773
    individuals#hate         : 0.5365
    groups#hate              : 0.4603
    race#offensive           : 0.2292
    race#hate                : 0.2857
    politics#offensive       : 0.2118
    politics#hate            : 0.2913


## Step 5: Save Results

In [7]:
import json
from pathlib import Path
import pandas as pd
import numpy as np

# Create output directory
output_dir = Path("/kaggle/working/outputs")
output_dir.mkdir(exist_ok=True)

# Convert numpy types
def convert_to_serializable(obj):
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Save
results = {'PhoBERT': convert_to_serializable(result)}
with open(output_dir / "results_datasetA_phobert.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Results saved to /kaggle/working/outputs/results_datasetA_phobert.json")
print(f"\n📊 PhoBERT Results:")
print(f"   Subset Accuracy: {result['subset_accuracy']:.4f}")
print(f"   Macro F1: {result['f1_macro']:.4f}")
print(f"   Micro F1: {result['f1_micro']:.4f}")


✅ Results saved to /kaggle/working/outputs/results_datasetA_phobert.json

📊 PhoBERT Results:
   Subset Accuracy: 0.5089
   Macro F1: 0.2586
   Micro F1: 0.5412


## Step 6: Save Model State Dict

Lưu state_dict để có thể load lại model sau này với đầy đủ attention weights (cho hate word extraction)

In [8]:
import torch
from pathlib import Path

# Create model checkpoint directory
checkpoint_dir = Path("/kaggle/working/checkpoints")
checkpoint_dir.mkdir(exist_ok=True)

# Save state_dict (bao gồm attention weights cho hate word extraction)
checkpoint_path = checkpoint_dir / "phobert_datasetA_state_dict.pt"

torch.save({
    'model_state_dict': model_phobert.model.state_dict(),
    'model_config': {
        'model_name': 'vinai/phobert-base',
        'num_labels': 11,
        'problem_type': 'multi_label_classification'
    },
    'training_info': {
        'dataset': 'A',
        'technique': 'Standard Training',
        'subset_accuracy': result['subset_accuracy'],
        'f1_macro': result['f1_macro'],
        'f1_micro': result['f1_micro']
    }
}, checkpoint_path)

print(f"✅ State dict saved to: {checkpoint_path}")
print(f"📦 File size: {checkpoint_path.stat().st_size / (1024**2):.2f} MB")
print("\n💡 Cách load lại:")
print("   checkpoint = torch.load('phobert_datasetA_state_dict.pt')")
print("   model.model.load_state_dict(checkpoint['model_state_dict'])")
print("   # Sau đó có thể dùng model.predict(..., extract_hate_words=True)")


✅ State dict saved to: /kaggle/working/checkpoints/phobert_datasetA_state_dict.pt
📦 File size: 515.09 MB

💡 Cách load lại:
   checkpoint = torch.load('phobert_datasetA_state_dict.pt')
   model.model.load_state_dict(checkpoint['model_state_dict'])
   # Sau đó có thể dùng model.predict(..., extract_hate_words=True)
